## first model

This is the first try (after the proof of concept) to create a model which is capable of detecting all three classes correctly.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 3)))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                98336     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 99,491
Trainable params: 99,491
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', datetime.now().strftime("%Y-%m-%dT%H-%M%-S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=50,
    callbacks=callbacks)

Epoch 1/50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
20/20 [==============================] - 2s 117ms/step - loss: 1.1253 - acc: 0.3125
Epoch 2/50
20/20 [==============================] - 2s 75ms/step - loss: 1.1077 - acc: 0.3150
Epoch 3/50
20/20 [==============================] - 2s 85ms/step - loss: 1.0716 - acc: 0.4500
Epoch 4/50
20/20 [==============================] - 2s 81ms/step - loss: 1.0781 - acc: 0.4575
Epoch 5/50
20/20 [==============================] - 2s 79ms/step - loss: 1.0311 - acc: 0.4975
Epoch 6/50
20/20 [==============================] - 2s 79ms/step - loss: 1.0590 - acc: 0.4450
Epoch 7/50
20/20 [==============================] - 1s 68ms/step - loss: 1.0023 - acc: 0.4950
Epoch 8/50
20/20 [==============================] - 1s 63ms/step - loss: 0.9560 - acc: 0.5725
Epoch 9/50
20/20 [==============================] - 2s 85ms/step - loss: 0.9318 - acc: 0.6250
Epoch 10/50
20/20 [==============================] - 1s 65m

In [6]:
model.evaluate_generator(test_generator)

[0.6272905841469765, 0.81333333]

In [7]:
model_path = join('models', 'symbol_classifier', 'first_model_data_augmentation.h5')
model.save(model_path)